### Step 0: Initialize knowledge about the characters

In [1]:
from src.character import characters

In [2]:
import re
import nltk
from nltk.parse import stanford

# Add the jar and model via their path (instead of setting environment variables):
#jar = '.\stanford-parser-full-2015-01-30\stanford-parser.jar'
#model = '.\stanford-parser-full-2015-01-30\stanford-parser.jar'

jar = '.\stanford-parser-full-2017-06-09\stanford-parser-3.8.0.jar'
model = '.\stanford-parser-full-2017-06-09\stanford-parser-3.8.0.jar'

parser = stanford.StanfordParser(model, jar, model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz', encoding='utf8')
dep_parser = stanford.StanfordDependencyParser(model, jar, model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz', encoding='utf8')


In [3]:
from src.curation import curation
curation(characters)

### Step 1: extracting dialogs and explicit speakers from raw text (deterministic method)

In [4]:
from src.explicit import get_annotated_lines

annotated_lines = get_annotated_lines(parser=parser, dep_parser=dep_parser)

[' said his lady to him one day, ']


LookupError: 

===========================================================================
NLTK was unable to find the java file!
Use software specific configuration paramaters or set the JAVAHOME environment variable.
===========================================================================

In [ ]:
from nltk.tree import Tree

t = Tree('ROOT', [Tree('SINV', [Tree('VP', [Tree('VBD', ['said'])]), Tree('NP', [Tree('NP', [Tree('PRP$', ['his']), Tree('NN', ['lady'])]), Tree('PP', [Tree('TO', ['to']), Tree('NP', [Tree('NP', [Tree('PRP', ['him'])]), Tree('NP', [Tree('CD', ['one']), Tree('NN', ['day'])]), Tree(',', [','])])])])])])
t.draw()

In [ ]:
t = [Tree('ROOT', [Tree('S', [Tree('VP', [Tree('VBG', ['returned']), Tree('NP', [Tree('PRP', ['she'])])]), Tree(':', [';'])])])]

t.draw()
